In [1]:
# !pip install opencv-python mediapipe sklearn matplotlib tensorflow==2.12.0``

In [14]:
!pip install imutils

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25854 sha256=cc55b123bd53c4145288554e9bff869f60b16e338d4bc16b1c72b4322b98341c
  Stored in directory: c:\users\lenovo\appdata\local\pip\cache\wheels\85\cf\3a\e265e975a1e7c7e54eb3692d6aa4e2e7d6a3945d29da46f2d7
Successfully built imutils


In [2]:
import cv2 
import mediapipe as mp
from matplotlib import pyplot as plt
import numpy as np
import os
import time
import imutils
import tensorflow as tf

In [5]:
mp_holistic = mp.solutions.holistic #Holistic Model
mp_drawing = mp.solutions.drawing_utils #Drawing utilities - mesh etc

In [6]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results # Image is same input image, return is a model object

In [7]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS)
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) 

In [9]:
def draw_styled_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                             mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=1, circle_radius=2),
                             mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=1, circle_radius=2)
                             )
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2, circle_radius=2)
                             )
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=2)
                             )
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                             )

In [10]:
cap = cv2.VideoCapture(0)
#Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
    
        #Read feed
        ret, frame = cap.read() #ret is return value which true or false, frame is the last frame captured
        
        #Make Detection
        image, results = mediapipe_detection(frame, holistic)
        
        
        #Draw landmarks
        draw_styled_landmarks(image, results)
        
        # show to screen
        resize = imutils.resize(image, width=720)
        cv2.imshow('OpenCV Feed', resize)

        # Break Gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [8]:
# pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)

In [9]:
# lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)

In [10]:
# rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

In [11]:
# face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)

In [11]:
def extract_keypoints(results):
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose,face,lh,rh])

In [13]:
# extract_keypoints(results).shape

In [14]:
DATA_PATH = os.path.join('MP_DATA')
actions = np.array(['hello', 'thanks', 'iloveyou'])
#This is number of videos
no_sequences = 30
#This is number of frames in a video
sequence_length = 30

In [20]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [21]:
flag = True

In [30]:
cap = cv2.VideoCapture(0)
#Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for action in actions:
        if not flag:
            break
        for sequence in range(no_sequence):
            if not flag:
                break
            for frame_num in range(sequence_length):

                #Read feed
                ret, frame = cap.read() #ret is return value which true or false, frame is the last frame captured

                #Make Detection
                image, results = mediapipe_detection(frame, holistic)

                #Draw landmarks
                draw_styled_landmarks(image, results)
                
                #Apply wait logic
                if frame_num == 0:
                    cv2.putText(image, 'STARTING COLLECTION', (120, 200),
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 1, cv2.LINE_AA)
                    cv2.putText(image, f'Collectinig frames for {action} Video Number {sequence}', (30, 10),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)
                    cv2.waitKey(1500)
                else:
                    cv2.putText(image, f'Collectinig frames for {action} Video number {sequence}', (15, 12),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints) #Path and array name is same
                
                # show to screen
                resize = imutils.resize(image, width=720)
                cv2.imshow('OpenCV Feed', resize)

                # Break Gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    print('q is pressed')
                    print('setting flag == False')
                    flag = False
                    break
#             if cv2.waitKey(10) & 0xFF == ord('w'):
#                 print('w is pressed')
#                 break
#         if cv2.waitKey(10) & 0xFF == ord('q'):
#             print('q is pressed')
#             break
    cap.release()
    cv2.destroyAllWindows()
    flag = True

In [25]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [26]:
label_map = {label:num for num,label in enumerate(actions)}

In [66]:
label_map

{'hello': 0, 'thanks': 1, 'iloveyou': 2}

In [27]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [28]:
actions

array(['hello', 'thanks', 'iloveyou'], dtype='<U8')

In [29]:
np.array(sequences).shape

(90, 30, 1662)

In [70]:
actions

array(['hello', 'thanks', 'iloveyou'], dtype='<U8')

In [71]:
np.array(labels).shape

(90,)

In [31]:
X = np.array(sequences)

In [32]:
X.shape

(90, 30, 1662)

In [34]:
y = to_categorical(labels).astype(int) # changing y to one-hot encoding

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.05)

In [36]:
X_train.shape

(85, 30, 1662)

In [37]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.callbacks import TensorBoard

In [38]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir = log_dir)

In [39]:
y_train.shape

(85, 3)

In [40]:
X_train.shape

(85, 30, 1662)

In [41]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [54]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [55]:
model.load_weights('action.h5')

In [111]:
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), 
#              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [180]:
model.fit(X_train, y_train, epochs=200)

Epoch 1/200
3/3 [==============================] - 5s 97ms/step - loss: 2.8334 - categorical_accuracy: 0.3412
Epoch 2/200
3/3 [==============================] - 0s 91ms/step - loss: 9.8997 - categorical_accuracy: 0.3529
Epoch 3/200
3/3 [==============================] - 0s 92ms/step - loss: 1.0535 - categorical_accuracy: 0.3412
Epoch 4/200
3/3 [==============================] - 0s 96ms/step - loss: 1.0919 - categorical_accuracy: 0.4000
Epoch 5/200
3/3 [==============================] - 0s 88ms/step - loss: 1.4677 - categorical_accuracy: 0.3647
Epoch 6/200
3/3 [==============================] - 0s 91ms/step - loss: 1.3466 - categorical_accuracy: 0.4941
Epoch 7/200
3/3 [==============================] - 0s 90ms/step - loss: 1.0465 - categorical_accuracy: 0.5294
Epoch 8/200
3/3 [==============================] - 0s 91ms/step - loss: 0.9818 - categorical_accuracy: 0.4118
Epoch 9/200
3/3 [==============================] - 0s 94ms/step - loss: 0.8916 - categorical_accuracy: 0.5529
Epoch 10/2

3/3 [==============================] - 0s 98ms/step - loss: 0.2670 - categorical_accuracy: 0.8706
Epoch 148/200
3/3 [==============================] - 0s 94ms/step - loss: 0.2248 - categorical_accuracy: 0.9059
Epoch 149/200
3/3 [==============================] - 0s 91ms/step - loss: 0.2223 - categorical_accuracy: 0.9529
Epoch 150/200
3/3 [==============================] - 0s 95ms/step - loss: 0.2292 - categorical_accuracy: 0.9059
Epoch 151/200
3/3 [==============================] - 0s 95ms/step - loss: 0.2498 - categorical_accuracy: 0.9294
Epoch 152/200
3/3 [==============================] - 0s 95ms/step - loss: 0.2834 - categorical_accuracy: 0.8941
Epoch 153/200
3/3 [==============================] - 0s 94ms/step - loss: 0.1942 - categorical_accuracy: 0.9176
Epoch 154/200
3/3 [==============================] - 0s 105ms/step - loss: 0.1911 - categorical_accuracy: 0.9412
Epoch 155/200
3/3 [==============================] - 0s 93ms/step - loss: 0.5796 - categorical_accuracy: 0.7529
Epoch

In [56]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 3)                 99        
                                                                 
Total params: 596,675
Trainable params: 596,675
Non-trai

In [57]:
res = model.predict(X_test)

1/1 [==============================] - 1s 738ms/step


In [58]:
actions[np.argmax(res[4])]

'hello'

In [59]:
actions[np.argmax(y_test[4])]

'iloveyou'

In [235]:
# model.save('actions.h5')

In [60]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [61]:
yhat = model.predict(X_test)

1/1 [==============================] - 0s 69ms/step


In [62]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [63]:
multilabel_confusion_matrix(ytrue,yhat)

array([[[2, 2],
        [0, 1]],

       [[3, 1],
        [0, 1]],

       [[2, 0],
        [3, 0]]], dtype=int64)

In [64]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num],-1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255),2, cv2.LINE_AA)
    return output_frame

In [ ]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

In [68]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

1/1 [==============================] - 0s 55ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 51ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 35ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 33ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 52ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 98ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 35ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 41ms/step
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 51ms/step
iloveyou
<class 'mediapip

1/1 [==============================] - 0s 62ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 62ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 34ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 88ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 51ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 58ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 33ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 34ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 30ms/step
hello
<class 'mediapipe.pyth